Este código carga el dataset completo, lo preprocesa aplicando las mismas transformaciones que se realizaron anteriormente, entrena el modelo optimizado de Random Forest con todos los datos y guarda el modelo entrenado en un archivo para su posterior uso.

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

# Cargar el dataset completo
train_data_full = pd.read_csv('desafio-de-prediccion-de-precios-de-portatiles/train.csv')

# Función de preprocesamiento
def preprocess_data(data):
    data = data.copy()
    
    # Convertir 'Ram' y 'Weight' a valores numéricos
    data['Ram'] = data['Ram'].str.replace('GB', '').astype(int)
    data['Weight'] = data['Weight'].str.replace('kg', '').astype(float)
    
    # Crear nuevas características para la resolución de pantalla
    resolution = data['ScreenResolution'].str.split('x', n=1, expand=True)
    data['X_resolution'] = resolution[0].str.extract(r'(\d+)').astype(int)
    data['Y_resolution'] = resolution[1].astype(int)
    
    # Extraer velocidad y serie del CPU
    data['Cpu_Speed'] = data['Cpu'].str.extract(r'(\d+\.\d+)GHz').astype(float)
    data['Cpu_Series'] = data['Cpu'].str.split().str[2:4].str.join(' ')
    
    # Transformar 'Memory' en múltiples columnas para HDD, SSD, etc.
    def extract_memory(memory, mem_type):
        if mem_type in memory:
            size_str = memory.split()[0].replace('GB', '').replace('TB', '000')
            try:
                return int(size_str)
            except ValueError:
                return 0
        return 0
    
    data['HDD'] = data['Memory'].apply(lambda x: extract_memory(x, 'HDD'))
    data['SSD'] = data['Memory'].apply(lambda x: extract_memory(x, 'SSD'))
    data['Flash_Storage'] = data['Memory'].apply(lambda x: extract_memory(x, 'Flash Storage'))
    data['Hybrid'] = data['Memory'].apply(lambda x: extract_memory(x, 'Hybrid'))
    
    # Crear dummies para 'Cpu_Series'
    data = pd.get_dummies(data, columns=['Cpu_Series'], drop_first=True)
    
    # Seleccionar las características relevantes
    features = ['Inches', 'Ram', 'Weight', 'X_resolution', 'Y_resolution', 'Cpu_Speed', 'HDD', 'SSD', 'Flash_Storage', 'Hybrid']
    features += [col for col in data.columns if 'Cpu_Series_' in col]
    
    X = data[features]
    y = data['Price_euros']
    
    return X, y

# Preprocesar los datos
X_full, y_full = preprocess_data(train_data_full)

# Entrenar el modelo de Random Forest optimizado con todos los datos de entrenamiento
optimized_rf = RandomForestRegressor(
    max_depth=None, 
    min_samples_leaf=1, 
    min_samples_split=2, 
    n_estimators=300,
    random_state=42
)

optimized_rf.fit(X_full, y_full)

# Guardar el modelo entrenado
joblib.dump(optimized_rf, 'optimized_random_forest_model.pkl')



['optimized_random_forest_model.pkl']

Este código hará lo siguiente:

Cargará los datos de test.csv.

Preprocesará los datos de prueba de la misma manera que los datos de entrenamiento.

Asegurará que las columnas del conjunto de datos de prueba coincidan con las del conjunto de datos de entrenamiento.

Usará el modelo entrenado para hacer predicciones.

Guardará las predicciones en un archivo CSV.

In [4]:
# Cargar el dataset de test
test_data = pd.read_csv('desafio-de-prediccion-de-precios-de-portatiles/test.csv')

# Función de preprocesamiento (idéntica a la usada anteriormente)
def preprocess_data(data):
    data = data.copy()
    
    # Convertir 'Ram' y 'Weight' a valores numéricos
    data['Ram'] = data['Ram'].str.replace('GB', '').astype(int)
    data['Weight'] = data['Weight'].str.replace('kg', '').astype(float)
    
    # Crear nuevas características para la resolución de pantalla
    resolution = data['ScreenResolution'].str.split('x', n=1, expand=True)
    data['X_resolution'] = resolution[0].str.extract(r'(\d+)').astype(int)
    data['Y_resolution'] = resolution[1].astype(int)
    
    # Extraer velocidad y serie del CPU
    data['Cpu_Speed'] = data['Cpu'].str.extract(r'(\d+\.\d+)GHz').astype(float)
    data['Cpu_Series'] = data['Cpu'].str.split().str[2:4].str.join(' ')
    
    # Transformar 'Memory' en múltiples columnas para HDD, SSD, etc.
    def extract_memory(memory, mem_type):
        if mem_type in memory:
            size_str = memory.split()[0].replace('GB', '').replace('TB', '000')
            try:
                return int(size_str)
            except ValueError:
                return 0
        return 0
    
    data['HDD'] = data['Memory'].apply(lambda x: extract_memory(x, 'HDD'))
    data['SSD'] = data['Memory'].apply(lambda x: extract_memory(x, 'SSD'))
    data['Flash_Storage'] = data['Memory'].apply(lambda x: extract_memory(x, 'Flash Storage'))
    data['Hybrid'] = data['Memory'].apply(lambda x: extract_memory(x, 'Hybrid'))
    
    # Crear dummies para 'Cpu_Series'
    data = pd.get_dummies(data, columns=['Cpu_Series'], drop_first=True)
    
    # Seleccionar las características relevantes
    features = ['Inches', 'Ram', 'Weight', 'X_resolution', 'Y_resolution', 'Cpu_Speed', 'HDD', 'SSD', 'Flash_Storage', 'Hybrid']
    features += [col for col in data.columns if 'Cpu_Series_' in col]
    
    X = data[features]
    
    return X

# Preprocesar los datos de test
X_test_full = preprocess_data(test_data)

# Cargar el modelo entrenado
optimized_rf = joblib.load('optimized_random_forest_model.pkl')

# Asegurarse de que las columnas de X_test_full coincidan con las de X_full
missing_cols = set(X_full.columns) - set(X_test_full.columns)
for col in missing_cols:
    X_test_full[col] = 0

X_test_full = X_test_full[X_full.columns]

# Hacer predicciones
predictions = optimized_rf.predict(X_test_full)

# Guardar las predicciones en un archivo CSV
test_data['Price_euros'] = predictions
test_data[['id', 'Price_euros']].to_csv('predictions.csv', index=False)



In [6]:
predictions=pd.read_csv('desafio-de-prediccion-de-precios-de-portatiles/predictions.csv')
predictions.head()

,id,Price_euros
0,181,2207.372478
1,708,984.316352
2,862,472.251638
3,1064,1378.962444
4,702,1445.930537


In [7]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           391 non-null    int64  
 1   Price_euros  391 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.2 KB


In [8]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [10]:
sample= pd.read_csv("desafio-de-prediccion-de-precios-de-portatiles/sample_submission.csv")
sample

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0
...,...,...
386,1162,258.0
387,1286,1085.0
388,336,298.0
389,885,1099.0


In [12]:
chequeator(predictions)

You're ready to submit!


NameError: name 'submission' is not defined

El score en la competicion de kaggle es 290,140
https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles/submissions
